In [1]:
# Indusries
# Job Categories
# Locations ==> Suburb (optional)
# Job Title ==> Do (10) categories (cleaning) ==> Count Vectorizer
# Work Type
# Dates <== missing
# Job Description

# =============================================
#### Dummies
# =============================================
# ==> Industries
# ==> Location
# ==> Job Title

# =============================================
#### CountVectorizer
# =============================================
# ==> Job Title ==> Experiences columns (Senior, Junior, Entry-level)


# Step 1. Do Linear Regression

In [2]:
####
# Step 0: graph salary distribution (neg, pos skew)
# ==> EDA:
#     Contract DS (140 - 200k)
#     Contract DA (45 - 60k)
#     Contract (25%) | Fulltime (75%)
#     Industries (20-30) ==> to (10-15)
#     Job Titles ==> (DS [Junior, Senior/Manager, just DS], 
#                     DA [Junior, Senior/Manager, just DA]
#                     DE [Junior, Senior/Manager, just DE])
#                 ==> to (12-15)
#     Categories (200) ==> to (20)

# Step 1: Regression Problems
# ==> issues: 
#     not good model
# ==> treat as classification problem


In [3]:
import pandas as pd
import numpy as np
import re

import config

# Folder
folder = './training/'
dest_folder = folder + 'Step5/'

target_file = folder + 'training_next_16052019.csv'
dest_file =  dest_folder + 'cleaned__training_16052019.csv'

In [4]:
frame = pd.read_csv(target_file, sep='|')
frame.head()

,content,industries,location,title,type_empl,url,wage
0,Over the last 2 years this business has been s...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Scientist Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/38892320?type=stan...,87575
1,The Data Architect is responsible for enterpri...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Architect Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/39015289?type=stan...,114181
2,About AFSA The Australian Financial Security A...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935314?type=stan...,87747
3,The Data Engineer (Cloud/Data Lake) will play ...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Engineer - Cloud / Data Lake Job in Brisb...,Full Time,https://www.seek.com.au/job/39015517?type=stan...,114181
4,About AFSA The Australian Financial Security A...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935330?type=stan...,87747


In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 7 columns):
content       1261 non-null object
industries    1261 non-null object
location      1261 non-null object
title         1261 non-null object
type_empl     1261 non-null object
url           1261 non-null object
wage          1261 non-null int64
dtypes: int64(1), object(6)
memory usage: 69.0+ KB


In [6]:
frame['title'].value_counts()

Business Analyst Job in Sydney - SEEK                                                             10
Senior Business Analyst Job in Sydney - SEEK                                                      10
Financial Analyst Job in Sydney - SEEK                                                             7
Data Analyst Job in Sydney - SEEK                                                                  7
Test Analyst Job in Brisbane - SEEK                                                                5
Senior Commercial Analyst Job in Sydney - SEEK                                                     5
Campaign Project Officer Job in Sydney - SEEK                                                      5
Data Analyst Job in Melbourne - SEEK                                                               5
Lecturer/Senior Lecturer in Cyber Security - Cryptography/Automation Job in Brisbane - SEEK        4
Software Engineer (NodeJS, React, AWS) Job in Sydney - SEEK                                

In [7]:
# Split Suburb out of content
def get_main_location(regex, sample):
#     print("In:", sample)
    result = re.search(regex, str(sample))

    if result:
#         print(result[0])
        result = result[0]
        result = result.replace(",","").lstrip().rstrip()
        return result
    elif len(str(sample)) > 3:
        return str(sample)
    return ''

# 
def get_suburb(regex, sample):
    result = re.sub(regex,"",str(sample))
    
    return result

In [8]:
frame['suburb'] = frame['location'].map(lambda row: get_suburb(r"(^[A-Za-z]+, )",row))
frame['location'] = frame['location'].map(lambda row: get_main_location(r"(^[A-Za-z]+, )",row))
frame['title'] = frame['title'].map(lambda row: row[:(row.find("Job")-1)])

In [9]:
frame['title'].value_counts()

Data Analyst                                                               15
Business Analyst                                                           14
Senior Business Analyst                                                    13
Test Analyst                                                                8
Financial Analyst                                                           8
Postdoctoral Research Fellow                                                7
Data Scientist                                                              7
Senior Data Scientist                                                       6
Senior Commercial Analyst                                                   5
Commercial Analyst                                                          5
Campaign Analyst                                                            5
Campaign Project Officer                                                    5
Software Engineer                                               

In [10]:
# Store all "keysearch" values in separate dataFrame.
# frame1 = []
# for value in fields:
#     result = frame[frame['title'] == value]
#     frame1.append(result)
# df = pd.concat(frame1)
# out_index = df.index.values
# df = df.reset_index(drop=True)
# df.head()
# print(df.shape)
# print("==============\n")
# frame.drop(out_index,inplace=True)
# frame.info()

In [11]:
# Round2: title include "keysearch" plus
# New dataFrame columns store the rest.
# titles = frame['title'].values
# new_titles = [''] * len(titles)
# title_left_over = [''] * len(titles)

# for i in fields:
#     for key, title in enumerate(titles):
#         if i in title:
#             # Push left over text to list
#             left_over = title.replace(i,"")
#             left_over = re.sub(r"([^a-zA-z\s])"," ",left_over)
#             left_over = re.sub(r"\s+"," ",left_over)

#             title_left_over[key] = left_over.lstrip().rstrip()
            
#             # Granted title as expected field
#             new_titles[key] = i
#         else:
#             title_left_over[key] = ''    
#             new_titles[key] = title
# df_title_lover = pd.DataFrame(title_left_over)
# frame['title'] = new_titles
# frame['left_over_title'] = title_left_over

In [12]:
titles = frame['title'].values
new_titles = frame['title'].values
experiences = [''] * len(titles)
title_left_over = [''] * len(titles)

for key, value in enumerate(new_titles):
    print("value: ", value)
    is_not_simple = True
    # Simple mapping level
    for exp in config.EXPERIENCE_LEVEL:
        if exp in value:       
            print("Exp in Exp_levels : ", exp,' ------- ' ,value)
            experiences[key] = exp
            title_left_over[key] = value
            new_titles[key] = value.replace(exp,"").rstrip().lstrip()            
            is_not_simple = False
            print("New title: ", new_titles[key])
            break
    
    # Dictionary mapping level
    print(config.TITLE_MAPPING_DICTIONARY)
    title_mapping_dictionary = config.TITLE_MAPPING_DICTIONARY
    if is_not_simple:
        for k_dict, v_dict in title_mapping_dictionary.items():
            for v_dict_item in v_dict:
                if v_dict_item in value:
                    print("Exp mapping dict : ", k_dict,' ------- ' ,value)
                    experiences[key] = k_dict
                    title_left_over[key] = value
                    new_titles[key] = value.replace(v_dict_item,"").rstrip().lstrip()
                    new_titles[key] = new_titles[key].replace(k_dict,"").rstrip().lstrip()
                    print("New title: ", new_titles[key])
                    break
                else:
                    experiences[key] = 'General'
    
    # Context mapping level as dictionary
    # For updating title only, NOT experiences
    context_mapping_dictionary = config.CONTEXT_MAPPING_DICTIONARY
    for k_dict, v_dict in context_mapping_dictionary.items():
        for v_dict_item in v_dict:
            if v_dict_item in value:
                print("[Title] mapping dict : ", k_dict,' ------- ' ,value)
                title_left_over[key] = value
#                 title_left_over[key] = value.replace(v_dict_item,".__.")
                new_titles[key] = k_dict
                break
        
    print("=================\n")
frame['title'] = new_titles
frame['experiences'] = experiences
frame['title_left_over'] = title_left_over

value:  Data Scientist
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Scientist  -------  Data Scientist

value:  Data Architect
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Applications Developer Enterprise Data Warehouse (EDW)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Data Engineer - Cloud / Data Lake
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mappin

value:  Senior Marketing Automation Specialist
Exp in Exp_levels :  Senior  -------  Senior Marketing Automation Specialist
New title:  Marketing Automation Specialist
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Senior Dairy Buyer - FMCG - $170k pkg
Exp in Exp_levels :  Senior  -------  Senior Dairy Buyer - FMCG - $170k pkg
New title:  Dairy Buyer - FMCG - $170k pkg
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Scientist  -------  Senior Dairy Buyer - FMCG - $170k pkg

value:  Reporting Analyst
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Jun

value:  BI Developer - SSAS
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Engineers  -------  BI Developer - SSAS

value:  Business Analyst
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Business Analyst

value:  Data Migration Specialist | South East | Dynamics 365
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Data Migration Specialist | South East | Dynamics 365

value:  Administration Support Officer - Traffic Management
{'Senior': ['Seni

value:  Personal Loan Broker
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Retail Buyer
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Scientist  -------  Retail Buyer

value:  Azure AD Support Consultant
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Mandate Compliance Senior Analyst
Exp in Exp_levels :  Senior  -------  Mandate Compliance Senior Analyst
New title:  Mandate Compliance  Analyst
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['a

[Title] mapping dict :  Data Analyst  -------  Business Development Manager - Data Analytics - Financial Services/Insurance
[Title] mapping dict :  Finance  -------  Business Development Manager - Data Analytics - Financial Services/Insurance

value:  BI/ Snr Consultant Role – Data and Analytics (Power Bi)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
Exp mapping dict :  Senior  -------  BI/ Snr Consultant Role – Data and Analytics (Power Bi)
New title:  BI/  Consultant Role – Data and Analytics (Power Bi)
[Title] mapping dict :  Data Analyst  -------  BI/ Snr Consultant Role – Data and Analytics (Power Bi)

value:  Performance & Data Analyst, National Corrections Statistics Group, Clerk...
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior

...

[Title] mapping dict :  Finance  -------  Commercial Finance Analyst

value:  Software Engineer (NodeJS, React, AWS)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Engineers  -------  Software Engineer (NodeJS, React, AWS)

value:  Business Analyst - Finance and Lending
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Business Analyst - Finance and Lending
[Title] mapping dict :  Finance  -------  Business Analyst - Finance and Lending

value:  Business Analyst | 6 Month Contract!
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior'

[Title] mapping dict :  Data Analyst  -------  Financial Crime Investigations Analyst
[Title] mapping dict :  Finance  -------  Financial Crime Investigations Analyst

value:  Team Leader - Capability Development Engineer, State Electronic Evidence...
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
Exp mapping dict :  Senior  -------  Team Leader - Capability Development Engineer, State Electronic Evidence...
New title:  Team er - Capability Development Engineer, State Electronic Evidence...

value:  Team Leader - Capability Development Engineer, State Electronic Evidence...
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
Exp mapping dict :  Senior  -------  Team Leader - Capability Development Engineer

Exp in Exp_levels :  Senior  -------  Senior Credit Underwriter
New title:  Credit Underwriter
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Customer Service Analyst 
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Customer Service Analyst 

value:  Treasury Officer
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Treasury Officer
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Juni

[Title] mapping dict :  Academic  -------  SKYHOPPER RESEARCH ASSOCIATE - PROJECT MANAGER

value:  Senior Medical Science Liaison
Exp in Exp_levels :  Senior  -------  Senior Medical Science Liaison
New title:  Medical Science Liaison
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Scientist  -------  Senior Medical Science Liaison

value:  Study Coordinator
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Software Developer - Research Application Development
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dic

Exp mapping dict :  Senior  -------  Marketing Manager - Cards Loyalty
New title:  Marketing  - Cards Loyalty

value:  Paid Search Manager
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
Exp mapping dict :  Senior  -------  Paid Search Manager
New title:  Paid Search
[Title] mapping dict :  Data Scientist  -------  Paid Search Manager

value:  Postdoctoral Research Fellow in Alzheimer’s Disease
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Academic  -------  Postdoctoral Research Fellow in Alzheimer’s Disease

value:  Postdoctoral Research Fellow in Alzheimer’s Disease
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'Gene

In [13]:
# frame['experiences'].value_counts()

In [14]:
frame['title'].value_counts()

Data Analyst                                                                     370
Academic                                                                         123
Engineers                                                                         90
Data Scientist                                                                    82
Account                                                                           79
Finance                                                                           61
Advisor Human Resources                                                            4
Project Delivery                                                                   4
Digital Marketing                                                                  4
Project                                                                            3
Partnership  - Life Insurance                                                      3
Administration Support Officer - Traffic Management              

In [15]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 10 columns):
content            1261 non-null object
industries         1261 non-null object
location           1261 non-null object
title              1261 non-null object
type_empl          1261 non-null object
url                1261 non-null object
wage               1261 non-null int64
suburb             1261 non-null object
experiences        1261 non-null object
title_left_over    1261 non-null object
dtypes: int64(1), object(9)
memory usage: 98.6+ KB


In [16]:
frame.head(50)

,content,industries,location,title,type_empl,url,wage,suburb,experiences,title_left_over
0,Over the last 2 years this business has been s...,Information & Communication Technology,Brisbane,Data Scientist,Full Time,https://www.seek.com.au/job/38892320?type=stan...,87575,CBD & Inner Suburbs,General,Data Scientist
1,The Data Architect is responsible for enterpri...,Information & Communication Technology,Brisbane,Data Architect,Full Time,https://www.seek.com.au/job/39015289?type=stan...,114181,CBD & Inner Suburbs,General,
2,About AFSA The Australian Financial Security A...,Information & Communication Technology,Brisbane,Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935314?type=stan...,87747,CBD & Inner Suburbs,General,
3,The Data Engineer (Cloud/Data Lake) will play ...,Information & Communication Technology,Brisbane,Engineers,Full Time,https://www.seek.com.au/job/39015517?type=stan...,114181,CBD & Inner Suburbs,General,Data Engineer - Cloud / Data Lake
4,About AFSA The Australian Financial Security A...,Information & Communication Technology,Brisbane,Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935330?type=stan...,87747,CBD & Inner Suburbs,General,
5,"This is a newly created role, dedicated to for...",Information & Communication Technology,Brisbane,Data Scientist,Full Time,https://www.seek.com.au/job/38890886?type=stan...,150000,CBD & Inner Suburbs,Senior,Senior Data Scientist
6,"​​​​​ About the team Big data, information and...",Consulting & Strategy,Brisbane,Data Analyst,Full Time,https://www.seek.com.au/job/38942934?type=stan...,210000,CBD & Inner Suburbs,General,Director of Big Data
7,The Opportunity Use Your Skills and Make an Im...,Science & Technology,Brisbane,Academic,Full Time,https://www.seek.com.au/job/39040919?type=stan...,102791,Western Suburbs & Ipswich,General,Research Scientist In Field Robotics
8,Department of Transport and Main Roads / Brisb...,Government & Defence,Brisbane,Data Scientist,Full Time,https://www.seek.com.au/job/38871761?type=stan...,88345,CBD & Inner Suburbs,Senior,Senior Data Scientist
9,Senior Analytics Lead - Claims | Cognos | Tabl...,Information & Communication Technology,Brisbane,Data Scientist,Full Time,https://www.seek.com.au/job/38935642?type=stan...,140000,Brisbane,Junior,"Claims Analytics Lead - Cognos, Tableau, Redsh..."


In [17]:
# experiences = [''] * len(titles)
# left_overs = frame['left_over_title'].values
# new_titles = frame['title'].values

# for key, value in enumerate(left_overs):
# #     print("Value: ", len(value), ' : ' ,value)
#     if len(value) > 1:        
#         is_simple = True
#         # Simple mapping levels
#         for exp in exp_levels:
#             if exp in value:
#                 print("Exp in Exp_levels : ", exp,' ------- ' ,value)
#                 experiences[key] = exp
#                 new_titles[key] = new_titles[key].replace(exp,"").rstrip()
#                 is_simple = False
#                 break
        
#         # Mapping exp base on dictionary
#         if is_simple:
#             for k_dict, v_dict in title_mapping_dictionary.items():
#                 for v_dict_list in v_dict:
#                     if v_dict_list in value:
#                         print("Exp mapping dict : ", k_dict,' ------- ' ,value)
#                         experiences[key] = k_dict
#                         new_titles[key] = new_titles[key].replace(k_dict,"").rstrip()
#                         break
                        
#     else:
#         # Incase not above               
#         experiences[key] = ''
# frame['exp'] = experiences
# frame['title'] = new_titles

In [18]:
# def get_exp(title):
#     for exp in experiences:
#         if exp in title:
#             title = title.replace(exp,"")
#             return title.lstrip().rstrip()
#     return ''
    
# frame['experience'] = frame['title'].map(get_exp)
# frame['experience'].value_counts()

In [19]:
# Turn type_empl to 2 sub-categories
wtype = frame['type_empl'].values
new_wtype = [''] * len(wtype)

for key, value in enumerate(wtype):
    if value in 'Full Time' or value in 'Part Time':
        new_wtype[key] = 'Permanent'
    else:
        new_wtype[key] = 'Contract'
frame['type_empl'] = new_wtype

# OUTPUT

In [20]:
output_df = pd.DataFrame(columns =frame.columns)

for key, value in enumerate(config.FIELDS):
    print(value)
    df_temp = frame[frame['title'] == value]
    
    print(df_temp.shape)
    output_df = pd.concat([output_df,df_temp])

Data Analyst
(370, 10)
Data Scientist
(82, 10)


In [21]:
output_df = output_df.reset_index(drop=True)
output_df.head()

,content,industries,location,title,type_empl,url,wage,suburb,experiences,title_left_over
0,"​​​​​ About the team Big data, information and...",Consulting & Strategy,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38942934?type=stan...,210000,CBD & Inner Suburbs,General,Director of Big Data
1,Public Safety Business Agency / Kedron Tempora...,Information & Communication Technology,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/39018343?type=stan...,85572,Northern Suburbs,General,Data Analyst
2,Fantastic working culture Flexible work enviro...,Information & Communication Technology,Brisbane,Data Analyst,Contract,https://www.seek.com.au/job/39007973?type=stan...,99961,CBD & Inner Suburbs,Senior,Senior Business Analyst
3,CBD Location 12 month contract Join this State...,Information & Communication Technology,Brisbane,Data Analyst,Contract,https://www.seek.com.au/job/38912693?type=stan...,174590,CBD & Inner Suburbs,General,Database Analyst
4,Salary packaging - to suit your personal and f...,Consulting & Strategy,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38921060?type=stan...,210000,CBD & Inner Suburbs,General,Director of Big Data


In [22]:
output_df.to_csv(dest_file,sep='|',index=False)